# Hands on Light Curve analysis

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import astropy
print('numpy:', np.__version__)
print('astropy:', astropy.__version__)

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits

To open the fits file we use `fits.open()` and just specify the filename as an argument:

In [ ]:
fits_file = fits.open('../1_read_data/data_test/run_05029747_DL3.fits')

We can retrieve some basic information on the  header data unit (HDU) by calling `.info()`:

In [ ]:
fits_file.info()

### Events

In [ ]:
events = fits_file['EVENTS']

In [ ]:
events.header

### PLOT SOURCE, POINTING and ON and OFF REGIONS

In [ ]:
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.wcs import WCS

First we get the pointing postion of the telescope

In [ ]:
pointing_pos =  SkyCoord( events.header.get('RA_PNT') *u.deg,   events.header.get('DEC_PNT') *u.deg )
pointing_pos

Then the postion of the source on the sky

In [ ]:
source_name = events.header.get('OBJECT')
print(source_name)
source_pos   =  SkyCoord.from_name(events.header.get('OBJECT'))
source_pos

As you can see the 2 position are not the same (Why?) 

let's compute the offset

In [ ]:
offset = source_pos.separation(pointing_pos)
offset.to( u.deg).value

We now compute the angle between the source and the pointing position.


In [ ]:
source_angle = pointing_pos.position_angle(source_pos).to(u.degree)
source_angle

Let's now compute the angles for the off positions

In [ ]:
off1_angle   = source_angle + 90*u.deg
off2_angle   = off1_angle   + 90*u.deg
off3_angle   = off2_angle   + 90*u.deg

Given the above angles, we now put the off positions on the same offset from the pointing that was used for the source

In [ ]:
off1_pos = pointing_pos.directional_offset_by(position_angle=off1_angle, separation=offset)  
off2_pos = pointing_pos.directional_offset_by(position_angle=off2_angle, separation=offset)  
off3_pos = pointing_pos.directional_offset_by(position_angle=off3_angle, separation=offset) 

We can now see the results

In [ ]:
fig, ax =  plt.subplots(figsize=(9,9))

ax = plt.subplot()


field_of_view = SphericalCircle( (pointing_pos.ra, pointing_pos.dec),
                    3.5 * u.deg,   edgecolor='black', facecolor='none')

ax.add_patch(field_of_view)

ax.scatter(pointing_pos.ra.value, pointing_pos.dec.value, marker='x', s=100, c='black', label ='Pointing'  ) 
ax.scatter(source_pos.ra.value, source_pos.dec.value, marker='*', s=100, c='red', label ='Source'  ) 
ax.scatter(off1_pos.ra.value, off1_pos.dec.value, marker='o', s=20, c='blue', label ='OFF 1'  ) 
ax.scatter(off2_pos.ra.value, off2_pos.dec.value, marker='o', s=20, c='orange', label ='OFF 2'  ) 
ax.scatter(off3_pos.ra.value, off3_pos.dec.value, marker='o', s=20, c='green', label ='OFF 3'  ) 
ax.legend(loc='upper right');

Or even better we can plot the On and Off regions

In [ ]:
fig, ax =  plt.subplots(figsize=(9,9))

ax = plt.subplot()




field_of_view = SphericalCircle( (pointing_pos.ra, pointing_pos.dec),
                    3.5 * u.deg,   edgecolor='black', facecolor='none')

radius = 0.1414213*u.deg

on  = SphericalCircle( (source_pos.ra, source_pos.dec),
                    radius,   edgecolor='black', facecolor='none')

off1 = SphericalCircle( (off1_pos.ra, off1_pos.dec),
                    radius,   edgecolor='black', facecolor='none')
off2 = SphericalCircle( (off2_pos.ra, off2_pos.dec),
                    radius,   edgecolor='black', facecolor='none')
off3 = SphericalCircle( (off3_pos.ra, off3_pos.dec),
                    radius,   edgecolor='black', facecolor='none')


ax.scatter(pointing_pos.ra.value, pointing_pos.dec.value, marker='x', s=100, c='black', label ='Pointing'  ) 
ax.scatter(source_pos.ra.value, source_pos.dec.value, marker='*', s=100, c='red', label ='Source'  ) 

ax.add_patch(field_of_view)

ax.add_patch(on)
ax.add_patch(off1)
ax.add_patch(off2)
ax.add_patch(off3)


ax.legend();

### PLOT AZIMUTH AND ALTITUDE DURING THE OBSERVATION

This is not necessary for the analysis, but it is interesting to see how the altitude and azimuth of the source evolves during the observation

In [ ]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

In [ ]:
ROM = EarthLocation( lat=events.header.get('GEOLAT')* u.deg, 
              lon=events.header.get('GEOLON') * u.deg, 
              height= events.header.get('ALTITUDE')*u.m)
print(ROM)

In [ ]:
start_time = Time( events.header.get('DATE-OBS') +'T'+events.header.get('TIME-OBS')) 
end_time   = Time( events.header.get('DATE-END') +'T'+events.header.get('TIME-END')) 
print(start_time)
print(end_time)

In [ ]:
dt = ( end_time - start_time ) /100
times        = []
source_altaz = []
for i in range(100):
    time        = start_time + dt*i
    altaz        = AltAz(obstime=time, location=ROM)
    i_source_altaz = source_pos.transform_to(altaz)
    times.append( time )
    source_altaz.append( i_source_altaz )
    

In [ ]:
all_alt = [  i.alt.value for i in source_altaz ]
all_az  = [  i.az.value  for i in source_altaz ]

In [ ]:
plt.plot(  all_az, all_alt)
plt.xlabel( "azimuth [deg.]")
plt.ylabel( "altitute [deg.]");

## GET EVENTS

We save the evtns as an Astropy Table

In [ ]:
events_table = Table( events.data )
events_table

We then plot them "on the Sky" using the information on the RA and DEC of each single event

In [ ]:
fig, ax =  plt.subplots(figsize=(9,9))

ax = fig.add_subplot()

ra =   -360*u.deg + events_table['RA'] * u.deg  

dec = events_table['DEC']*u.deg

ax.scatter(ra, dec, s=0.1)


radius = 0.1414213*u.deg

on  = SphericalCircle( (source_pos.ra, source_pos.dec),
                    radius,   edgecolor='black', facecolor='none')

off1 = SphericalCircle( (off1_pos.ra, off1_pos.dec),
                    radius,   edgecolor='black', facecolor='none')
off2 = SphericalCircle( (off2_pos.ra, off2_pos.dec),
                    radius,   edgecolor='black', facecolor='none')
off3 = SphericalCircle( (off3_pos.ra, off3_pos.dec),
                    radius,   edgecolor='black', facecolor='none')


ax.scatter(pointing_pos.ra.value, pointing_pos.dec.value, marker='x', s=100, c='black', label ='Pointing'  ) 
ax.scatter(source_pos.ra.value, source_pos.dec.value, marker='*', s=100, c='red', label ='Source'  ) 

ax.add_patch(on)
ax.add_patch(off1)
ax.add_patch(off2)
ax.add_patch(off3)


ax.legend();

As expected we have more events near the Source!

As one can see from the OFF regions, some of these events are not gamma-rays, but background

Rember that in the Off region we should not expect any gamma-ray

### EVENTS SELECTION

We now select the events in the 4 regions: 1 ON and 3 OFF

In [ ]:
ra = events_table['RA'] 
dec = events_table['DEC']

radius = 0.1414213*u.deg 

cond_on = []
for i_ra,i_dec in zip(ra,dec):
    position_event = SkyCoord(  i_ra * u.deg, i_dec * u.deg, frame='icrs')

    cond_on.append( source_pos.separation(position_event) <= radius )
    
cond_off1 = []
for i_ra,i_dec in zip(ra,dec):
    position_event = SkyCoord(  i_ra * u.deg, i_dec * u.deg, frame='icrs')

    cond_off1.append( off1_pos.separation(position_event) <= radius )
    
cond_off2 = []
for i_ra,i_dec in zip(ra,dec):
    position_event = SkyCoord(  i_ra * u.deg, i_dec * u.deg, frame='icrs')

    cond_off2.append( off2_pos.separation(position_event) <= radius )
    

cond_off3 = []
for i_ra,i_dec in zip(ra,dec):
    position_event = SkyCoord(  i_ra * u.deg, i_dec * u.deg, frame='icrs')

    cond_off3.append( off3_pos.separation(position_event) <= radius )
    
    

Let's plot the events we have selected

In [ ]:
fig, ax =  plt.subplots(figsize=(9,9))

ax = fig.add_subplot()

ra = events_table[cond_on]['RA'] - 360
dec = events_table[cond_on]['DEC']
ax.scatter(ra, dec, s=0.5)

ra = events_table[cond_off1]['RA'] - 360
dec = events_table[cond_off1]['DEC']
ax.scatter(ra, dec, s=0.5, c ='black')

ra = events_table[cond_off2]['RA'] - 360
dec = events_table[cond_off2]['DEC']
ax.scatter(ra, dec, s=0.5,  c ='black')

ra = events_table[cond_off3]['RA'] - 360
dec = events_table[cond_off3]['DEC']
ax.scatter(ra, dec, s=0.5,  c ='black')



ax.scatter(pointing_pos.ra.value, pointing_pos.dec.value, marker='x', s=100, c='black', label ='Pointing'  ) 
ax.scatter(source_pos.ra.value, source_pos.dec.value, marker='*', s=100, c='red', label ='Source'  ) 

radius = 0.1414213*u.deg

on  = SphericalCircle( (source_pos.ra, source_pos.dec),
                    radius,   edgecolor='black', facecolor='none')

off1 = SphericalCircle( (off1_pos.ra, off1_pos.dec),
                    radius,   edgecolor='black', facecolor='none')
off2 = SphericalCircle( (off2_pos.ra, off2_pos.dec),
                    radius,   edgecolor='black', facecolor='none')
off3 = SphericalCircle( (off3_pos.ra, off3_pos.dec),
                    radius,   edgecolor='black', facecolor='none')


ax.add_patch(on)
ax.add_patch(off1)
ax.add_patch(off2)
ax.add_patch(off3)



ax.legend();

### Effective Area

for computing the Light Curve we need the Total Effective Area

We will compute the Light Curbe for E > 300 GeV

In [ ]:
en_th = 0.3 # TeV

We load the Effctive Area from the fits file

In [ ]:
effective_area = fits_file['EFFECTIVE AREA']

In [ ]:
effective_area.header

It's better to work with an Astropy table

In [ ]:
effective_area = Table( effective_area.data)

We get from the table the enrgy bins and the value ofthe Area for each bin

In [ ]:
theta_low  = np.array( effective_area['THETA_LO'] )[0]
theta_high = np.array( effective_area['THETA_HI'] )[0]

en_low     = np.array( effective_area['ENERG_LO'] )[0]
en_high    = np.array( effective_area['ENERG_HI'] )[0]
en_center = np.sqrt( en_low * en_high )
delta_E  = (en_high - en_low)

eff_area   = np.array( effective_area['EFFAREA'] )[0][0]



Remove bad bins, i.e. those with Aeff = 0

In [ ]:
cond = eff_area >0
en_center = en_center[cond]
en_low    = en_low[cond]
en_high   = en_high[cond]
delta_E    = delta_E[cond]

eff_area  = eff_area[cond]

In [ ]:
fig, ax = plt.subplots(figsize=(10,6),nrows=1, ncols=1)

ax.scatter(en_center,  eff_area)


ax.set_xlabel( ' Energy / TeV')
ax.set_ylabel( ' Area / m^2')
ax.set_xscale('log')
ax.set_yscale('log')

We must integrate the affective area from the energy threshold up to the highest energy bin

For this integratyion we will use the function `integrate` of `scipy`

In [ ]:
import scipy.integrate as integrate

In [ ]:
max_en = en_high[-1]
total_effective_area = integrate.quad( lambda x: np.interp(x ,  en_center, eff_area), en_th, max_en)[0] * u.m**2 * u.TeV
total_effective_area

### COMPUTE EXCESS

We are now ready to compute the ON counts, OFF counts and the excess in each time bin

First we define how many bins we want

In [ ]:
n_time_bins = 12

Now we compute the bins over the whole observation time

In [ ]:
start_time = events.header.get('TSTART') 
end_time   = events.header.get('TSTOP') 

dt = ( end_time - start_time ) /n_time_bins
times_arr        = []
source_altaz = []
for i in range(n_time_bins):
    time        = start_time + dt*i
    times_arr.append( time )
    
times_arr = np.array(times_arr)

time_center = (times_arr[:-1] + times_arr[1:])/2
delta_t     =  (-times_arr[:-1] + times_arr[1:])

In [ ]:
excess = [] 
err_excess = []

noff_list = []
non_list  = []

en_th = 0.3 

for i_time_low, i_time_high in zip( times_arr[:-1], times_arr[1:]):
    
    cond_en_th = events_table['ENERGY'] > en_th
    
    cond_low  = events_table['TIME'] >= i_time_low
    cond_high = events_table['TIME'] < i_time_high
    cond_en   = cond_low* cond_high
    
    non    = np.sum( cond_en*cond_on * cond_en_th)
    
    noff   = np.sum( cond_en*cond_off1 * cond_en_th) +\
             np.sum( cond_en*cond_off2 * cond_en_th) +\
             np.sum( cond_en*cond_off3 * cond_en_th) 

    
    noff_list.append(noff)
    non_list.append(non)
    
    excess.append( non - noff/3 )
    err_excess.append( np.sqrt( non + (1/3)**2*noff) )
    
excess     = np.array(excess)
err_excess = np.array(err_excess)

non_list   = np.array(non_list)
noff_list   = np.array(noff_list)

Why did I put in the loop `non - noff/3` and not another number like for instance  `non - noff/5` ?

Total Non events in each energy bin

In [ ]:
print(non_list)

Total Noff events in each energy bin

In [ ]:
print(noff_list)

Excess in each energy bin

In [ ]:
print(excess)

Excercise
- Compute the Li&Ma significance for each energy bin using the value of Non and Noff computed (remeber that we are also using $\alpha$ = 1/3)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6),nrows=1, ncols=1)


ax.errorbar(x=time_center-start_time, y=excess, yerr=err_excess , xerr = delta_t/2 , c='black', capsize=2, fmt='o')

ax.set_ylim( [0, None])
ax.set_xlabel( ' Time from START / seconds')

ax.set_ylabel( ' Excess')

### LIGHT CURVE

We are now finally ready to compute the  LC in each time bin

In [ ]:
flux = excess/( total_effective_area)/  ( delta_t *u.second)  * ( max_en- en_th) *u.TeV

flux_err = err_excess/( total_effective_area)/  ( delta_t *u.second) * ( max_en- en_th) *u.TeV



We can now plot the flux in energy

For reference we will also print as a horizontal line the flux we should expect from the Crab Nebula

In [ ]:
fig, ax = plt.subplots(figsize=(10,6),nrows=1, ncols=1)


ax.errorbar(x=time_center-start_time, 
            y=flux.to( 1/(  u.cm**2 * u.s) ).value / 1e-9 , 
            yerr = flux_err.to( 1/( u.cm**2 * u.s) ).value  / 1e-9 , 
            xerr = delta_t/2 , c='black', capsize=2, fmt='o')



ax.hlines( 0.12, np.min(time_center-start_time), np.max(time_center-start_time) , label='Reference Light Curve')


ax.set_ylim( [0, None])
ax.set_xlabel( ' Time from START / seconds')

ax.set_ylabel( ' Flux > 300 GeV   / cm^2 s / 1e-9')
ax.legend();